In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
data=pd.read_csv("C:\\Users\\shali\\Downloads\\loan_information.csv")

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
# Formation of new column 'status' according to their account number

In [ ]:
data['account_number'].fillna("U",inplace=True)

In [ ]:
data.head()

In [ ]:
loan_data=[]
for i in data.account_number.values:
    if i=="U":
        loan_data.append("no")
    else:
        loan_data.append("yes")

loan_data

In [ ]:
data['status']=loan_data

In [ ]:
data.head()

In [ ]:
#dropping unwanted columns

In [ ]:
data=data.drop(['account_number','enterprise_id','hub_id', 'enterprise_id.1', 'disbursement_month','loan_id','customer_id', 'screening_date','hub_id', 'business_sector',],axis=1)

In [ ]:
data.head()

In [ ]:
cols = ['ManagAgeui_APP','total_disbursement_amount']
data = data.drop(columns=cols, axis=1)

In [ ]:
#Checking for null values

In [ ]:
data.isna().sum()

In [ ]:
#Filling all the categorical columns with mode and numerical columns with mean

In [ ]:
data["product_code"].fillna(data["product_code"].mode()[0],inplace=True)

In [ ]:
data["business_type"].fillna(data["business_type"].mode()[0],inplace=True)

In [ ]:
data["business_activity"].fillna(data["business_activity"].mode()[0],inplace=True)

In [ ]:
data["BusinFormalityOfTheBusinessui"].fillna(data["BusinFormalityOfTheBusinessui"].mode()[0],inplace=True)

In [ ]:
data["ManagCBscoreui_APP"].fillna(data["ManagCBscoreui_APP"].mean(),inplace=True)

In [ ]:
data.head()

In [ ]:
data.isna().sum()

In [ ]:
#rechanging the values of the column 'product code'

In [ ]:
Product=[]
for i in data.product_code.values:
    if i[-1]=="U":
        Product.append("U")
    else:
        Product.append("S")
        
Product

In [ ]:
data['Product_code'] = Product

In [ ]:
data=data.drop('product_code', axis='columns')

In [ ]:
data.info()

In [ ]:
#Label encoding the target column 'status'

In [ ]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()

data['status']=le.fit_transform(data['status'])

In [ ]:
#data visualisation

In [ ]:
sns.countplot(x= 'business_type',hue='status', data=data)

In [ ]:
data.boxplot(column='ManagCBscoreui_APP')

In [ ]:
data['ManagCBscoreui_APP'].hist(bins=20)

In [ ]:
data.head()

In [ ]:
plt.figure(figsize=(8,4))
sns.countplot(x= 'BusinFormalityOfTheBusinessui',hue='status', data=data)

In [ ]:
plt.figure(figsize=(8,4))
sns.countplot(data['BusinFormalityOfTheBusinessui'], palette='twilight')

In [ ]:
#One hot encoding to all the categorical columns by getting dummies

In [ ]:
productCode = pd.get_dummies(data['Product_code'], drop_first=True)
productCode.head()

In [ ]:
businessType = pd.get_dummies(data['business_type'], drop_first=True)
businessType.head()

In [ ]:
businessActivity = pd.get_dummies(data['business_activity'], drop_first=True) 
businessActivity.head()

In [ ]:
businessActivity=businessActivity.drop(['Job Work and sales','Whole Sale'], axis=1)

In [ ]:
BusinFormality = pd.get_dummies(data['BusinFormalityOfTheBusinessui'], drop_first=True)
BusinFormality.head()

In [ ]:
#concatinating dummies and removing the previous columns

In [ ]:
df = pd.concat([data, productCode, businessType, businessActivity, BusinFormality], axis = 1)
df

In [ ]:
df=df.drop(['Product_code','business_type','business_activity','BusinFormalityOfTheBusinessui'], axis=1)

In [ ]:
df.columns

In [ ]:
#separating x(independent) and y(dependent) columns

In [ ]:
x=df.drop(["status"],axis=1)

In [ ]:
x.head()

In [ ]:
x.columns

In [ ]:
y=data['status']

In [ ]:
y.head()

In [ ]:
#normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
scaler=MinMaxScaler()
scaler.fit_transform(x)

In [ ]:
#standardization

In [ ]:
from sklearn.preprocessing import StandardScaler 
scaler=StandardScaler()
scaler.fit_transform(x)

In [ ]:
#Splitting the data into training and testing data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
#Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf=LogisticRegression(random_state=0,max_iter=1000).fit(X_train,Y_train)

In [ ]:
predictions= clf.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print('accuracy:%.3f' %accuracy_score(Y_test, predictions))
print('recall:%.3f' %recall_score(Y_test, predictions))
print('precision:%.3f' %precision_score(Y_test, predictions))


In [ ]:
#Hyperparameter tuning for Logistic regression

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
clfier = GridSearchCV(cv=None,
       estimator=LogisticRegression(C=1.0, intercept_scaling=1, dual=False, fit_intercept=True,
          penalty='l2', tol=0.0001, max_iter=1000),
          n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, verbose=0)
grid_s=clfier.fit(X_train,Y_train)
Accuracy= grid_s.best_score_
Accuracy

In [ ]:
#decision tree 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DTClassifier=DecisionTreeClassifier(criterion="entropy",random_state=0)
DTClassifier.fit(X_train,Y_train)

In [ ]:
y_pred= DTClassifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print('accuracy:%.3f' %accuracy_score(Y_test, y_pred ))
print('recall:%.3f' %recall_score(Y_test, y_pred))
print('precision:%.3f' %precision_score(Y_test, y_pred))

In [ ]:
#Hyperparameter tuning for =Decision tree classifier

In [ ]:
clfier_dt = GridSearchCV(cv=3,
       estimator=DecisionTreeClassifier(),
          n_jobs=-1,
       param_grid={'min_samples_split': np.arange(2, 10), 'min_samples_leaf': 
np.arange(.05, .2), 'max_leaf_nodes': np.arange(2, 30)},
       pre_dispatch='2*n_jobs', refit=True, verbose=0)
grid_s=clfier_dt.fit(X_train,Y_train)
Accuracy= grid_s.best_score_
Accuracy

In [ ]:
#naive_bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
NBClassifier = GaussianNB()
NBClassifier.fit(X_train,Y_train)

In [ ]:
pred=NBClassifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print('accuracy:%.3f' %accuracy_score(Y_test, pred ))
print('recall:%.3f' %recall_score(Y_test, pred))
print('precision:%.3f' %precision_score(Y_test, pred))

In [ ]:
#Hyperparameter tuning for naive bays

In [ ]:
import numpy as np
clfier_dt = GridSearchCV(cv=10,
       estimator=GaussianNB(),
          n_jobs=-1,
       param_grid={'var_smoothing': np.logspace(0,-9, num=100)}, verbose=1,)
grid_s=clfier_dt.fit(X_train,Y_train)
Accuracy= grid_s.best_score_
Accuracy

In [ ]:
#random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,Y_train)

In [ ]:
predi=rf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print('accuracy:%.3f' %accuracy_score(Y_test, predi))
print('recall:%.3f' %recall_score(Y_test, predi))
print('precision:%.3f' %precision_score(Y_test, predi))

In [ ]:
#Hyperparameter tuning for Random forest

In [ ]:
rfbase = RandomForestClassifier(n_jobs = 3, max_features = 'auto', n_estimators = 10, bootstrap=False)


clfier_dt = GridSearchCV(cv=10,
       estimator=rfbase, 
       param_grid= {
    'n_estimators': [200,500],
    'max_features': [.5,.7],
    'bootstrap': [False, True],
    'max_depth':[3,6]
})

grid_s=clfier_dt.fit(X_train,Y_train)
Accuracy= grid_s.best_score_
Accuracy

In [ ]:
#KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,Y_train)

In [ ]:
predic=knn.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print('accuracy:%.3f' %accuracy_score(Y_test, predic))
print('recall:%.3f' %recall_score(Y_test, predic))
print('precision:%.3f' %precision_score(Y_test, predic))

In [ ]:
#Hyperparameter tuning for KNN

In [ ]:
clfier_dt = GridSearchCV(
       estimator=KNeighborsClassifier(n_neighbors=3),
          n_jobs=-1,
       param_grid={'n_neighbors': [1, 3, 5, 7, 11],'weights':['uniform','distance'],'metric':['euclidean','manhattan']},
       pre_dispatch='2*n_jobs', refit=True, verbose=0)
grid_s=clfier_dt.fit(X_train,Y_train)
Accuracy= grid_s.best_score_
Accuracy

In [ ]:
#xgboost

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, Y_train)

In [ ]:
y_predict = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print('accuracy:%.3f' %accuracy_score(Y_test, y_predict ))
print('recall:%.3f' %recall_score(Y_test, y_predict))
print('precision:%.3f' %precision_score(Y_test, y_predict))

In [ ]:
#Hyperparameter tuning for XG boost

In [ ]:
clfier_dt = GridSearchCV(
       estimator=XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=5,min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27),
       param_grid={'max_depth':[3],
 'min_child_weight':[3]},n_jobs=4, cv=5)

grid_s=clfier_dt.fit(X_train,Y_train)
Accuracy= grid_s.best_score_
Accuracy

In [ ]:
#Confusion matrix and roc auc curve for Logistic regression

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, predictions)
cm

In [ ]:
sns.heatmap(cm, cmap="Reds", annot= True)
plt.title('Confusion Matrix: Logistic Regression')
plt.xlabel('predicted class')
plt.ylabel('actual class')

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

p_predictions = clf.predict_proba(X_test)
logistic_fpr, logistic_tpr, threshold = roc_curve(Y_test, p_predictions[:, 1])
auc_logistic = auc(logistic_fpr, logistic_tpr)

plt.figure(figsize=(5, 5), dpi=100)
plt.plot(logistic_fpr, logistic_tpr, marker='_', label='logistic (auc= %0.3f)' % auc_logistic)
         
plt.xlabel('False positive rate -->')
plt.ylabel('True positive rate -->')
 
plt.legend()
         
plt.show()

roc_auc_score(Y_test, p_predictions[:, 1])

In [ ]:
#Confusion matrix and roc auc curve for Decision tree

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
cmatrix = confusion_matrix(Y_test, y_pred)
cmatrix

In [ ]:
sns.heatmap(cmatrix, cmap="Reds", annot= True)
plt.title('Confusion Matrix: Decision Tree')
plt.xlabel('predicted class')
plt.ylabel('actual class')

In [ ]:
p_y_pred = DTClassifier.predict_proba(X_test)
Dtree_fpr, Dtree_tpr, threshold = roc_curve(Y_test, p_y_pred[:, 1])
auc_Dtree = auc(Dtree_fpr, Dtree_tpr)

plt.figure(figsize=(5, 5), dpi=100)
plt.plot(Dtree_fpr, Dtree_tpr, marker='_', label='Decision tree (auc= %0.3f)' % auc_Dtree)
         
plt.xlabel('False positive rate -->')
plt.ylabel('True positive rate -->')
 
plt.legend()
         
plt.show()

roc_auc_score(Y_test, p_y_pred[:, 1])

In [ ]:
#Confusion matrix and roc auc curve for Naive bayes

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, pred)
confusion_matrix

In [ ]:
sns.heatmap(confusion_matrix, cmap="Reds", annot= True)
plt.title('Confusion Matrix: Naive Bayes')
plt.xlabel('predicted class')
plt.ylabel('actual class')

In [ ]:
p_pred = NBClassifier.predict_proba(X_test)
Nbays_fpr, Nbays_tpr, threshold = roc_curve(Y_test, p_pred[:, 1])
auc_Nbays = auc(Nbays_fpr, Nbays_tpr)

plt.figure(figsize=(5, 5), dpi=100)
plt.plot(Nbays_fpr, Nbays_tpr, marker='_', label='Nbays (auc= %0.3f)' % auc_Nbays)
         
plt.xlabel('False positive rate -->')
plt.ylabel('True positive rate -->')
 
plt.legend()
         
plt.show()

roc_auc_score(Y_test, p_pred[:, 1])

In [ ]:
#Confusion matrix and roc auc curve for Random forest

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(Y_test, predi)
conf_matrix

In [ ]:
sns.heatmap(conf_matrix, cmap="Reds", annot= True)
plt.title('Confusion Matrix: Random forest')
plt.xlabel('predicted class')
plt.ylabel('actual class')

In [ ]:
p_predi = rf.predict_proba(X_test)
Rforest_fpr, Rforest_tpr, threshold = roc_curve(Y_test, p_predi[:, 1])
auc_Rforest = auc(Rforest_fpr, Rforest_tpr)

plt.figure(figsize=(5, 5), dpi=100)
plt.plot(Rforest_fpr, Rforest_tpr, marker='_', label='Rforest (auc= %0.3f)' % auc_Rforest)
         
plt.xlabel('False positive rate -->')
plt.ylabel('True positive rate -->')
 
plt.legend()
         
plt.show()

roc_auc_score(Y_test, p_predi[:, 1])

In [ ]:
#Confusion matrix and roc auc curve for KNN

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
confu_matrix = confusion_matrix(Y_test, predic)
confu_matrix

In [ ]:
sns.heatmap(confu_matrix, cmap="Reds", annot= True)
plt.title('Confusion Matrix: KNN')
plt.xlabel('predicted class')
plt.ylabel('actual class')

In [ ]:
p_predic = knn.predict_proba(X_test)
KNN_fpr, KNN_tpr, threshold = roc_curve(Y_test, p_predic[:, 1])
auc_KNN = auc(KNN_fpr, KNN_tpr)

plt.figure(figsize=(5, 5), dpi=100)
plt.plot(KNN_fpr, KNN_tpr, marker='_', label='KNN (auc= %0.3f)' % auc_KNN)
         
plt.xlabel('False positive rate -->')
plt.ylabel('True positive rate -->')
 
plt.legend()
         
plt.show()

roc_auc_score(Y_test, p_predic[:, 1])

In [ ]:
#Confusion matrix and roc auc curve for Xgboost

In [ ]:
p_y_predict = model.predict_proba(X_test)
Xgboost_fpr, Xgboost_tpr, threshold = roc_curve(Y_test, p_y_predict[:, 1])
auc_Xgboost = auc(Xgboost_fpr, Xgboost_tpr)

plt.figure(figsize=(5, 5), dpi=100)
plt.plot(Xgboost_fpr, Xgboost_tpr, marker='.', label='Xgboost (auc= %0.3f)' % auc_Xgboost)
         
plt.xlabel('False positive rate -->')
plt.ylabel('True positive rate -->')
 
plt.legend()
         
plt.show()

roc_auc_score(Y_test, p_y_pred[:, 1])

In [ ]:
import pickle
with open('model_pickle.pkl', 'wb') as file:
    pickle.dump(clf,file)